In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from os import environ
from citrination_client import CitrinationClient
from citrination_client import *
from pypif import pif
from pypif.obj import *
import csv
import pandas as pd

/Users/Prabudhya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
dat_t= pd.read_pickle('elastic_modulus_temperature_with_features.pkl')

In [3]:
print(dat_t.shape)
dat_t.head()

(939, 143)


,Formula,Elastic Modulus (GPa),Temperature (K),Crystallinity,Reference,composition,minimum Number,maximum Number,range Number,mean Number,...,maximum SpaceGroupNumber,range SpaceGroupNumber,mean SpaceGroupNumber,avg_dev SpaceGroupNumber,mode SpaceGroupNumber,mean simul. packing efficiency,mean abs simul. packing efficiency,dist from 1 clusters |APE| < 0.010,dist from 3 clusters |APE| < 0.010,dist from 5 clusters |APE| < 0.010
0,SiC,402.0,293.0,Polycrystalline,Effects of Environment on the Mechanical Behav...,"(Si, C)",6.0,14.0,8.0,10.0,...,227.0,33.0,210.5,16.5,194.0,0.012098,0.017801,0.041595,0.087671,0.172624
1,SiC,382.0,1273.0,Polycrystalline,Effects of Environment on the Mechanical Behav...,"(Si, C)",6.0,14.0,8.0,10.0,...,227.0,33.0,210.5,16.5,194.0,0.012098,0.017801,0.041595,0.087671,0.172624
2,SiC,375.0,1523.0,Polycrystalline,Effects of Environment on the Mechanical Behav...,"(Si, C)",6.0,14.0,8.0,10.0,...,227.0,33.0,210.5,16.5,194.0,0.012098,0.017801,0.041595,0.087671,0.172624
3,SiC,372.0,1623.0,Polycrystalline,Effects of Environment on the Mechanical Behav...,"(Si, C)",6.0,14.0,8.0,10.0,...,227.0,33.0,210.5,16.5,194.0,0.012098,0.017801,0.041595,0.087671,0.172624
4,AlN,305.0,296.0,Polycrystalline,"Engineering Properties of Nitrides, S. Hampshi...","(Al, N)",7.0,13.0,6.0,10.0,...,225.0,31.0,209.5,15.5,194.0,-0.020239,0.029842,0.037216,0.129066,0.191011


Lets run two machine learning algorithms on this:
- SGD Regressor
- Random Forest
Following the examples over 
<a hyperref= "https://nbviewer.jupyter.org/github/hackingmaterials/matminer_examples/blob/master/notebooks/intro_predicting_bulk_modulus.ipynb">here</a>

## Split the dataset

In [25]:
dat_t_sgd= dat_t.copy() 

In [26]:
na_values= dat_t_sgd[dat_t_sgd.isnull().any(1)]
print(na_values.shape)
na_values

(64, 143)


,Formula,Elastic Modulus (GPa),Temperature (K),Crystallinity,Reference,composition,minimum Number,maximum Number,range Number,mean Number,...,maximum SpaceGroupNumber,range SpaceGroupNumber,mean SpaceGroupNumber,avg_dev SpaceGroupNumber,mode SpaceGroupNumber,mean simul. packing efficiency,mean abs simul. packing efficiency,dist from 1 clusters |APE| < 0.010,dist from 3 clusters |APE| < 0.010,dist from 5 clusters |APE| < 0.010
26,L2O3,178.3,293.0,Polycrystalline,Elastic Properties of Polycrystalline Thulium ...,"(L0+, O)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,L2O3,176.5,355.0,Polycrystalline,Elastic Properties of Polycrystalline Thulium ...,"(L0+, O)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,L2O3,175.1,419.0,Polycrystalline,Elastic Properties of Polycrystalline Thulium ...,"(L0+, O)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,L2O3,173.4,500.0,Polycrystalline,Elastic Properties of Polycrystalline Thulium ...,"(L0+, O)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,L2O3,172.0,581.0,Polycrystalline,Elastic Properties of Polycrystalline Thulium ...,"(L0+, O)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,L2O3,170.6,645.0,Polycrystalline,Elastic Properties of Polycrystalline Thulium ...,"(L0+, O)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,L2O3,169.7,708.0,Polycrystalline,Elastic Properties of Polycrystalline Thulium ...,"(L0+, O)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,L2O3,168.4,790.0,Polycrystalline,Elastic Properties of Polycrystalline Thulium ...,"(L0+, O)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,L2O3,167.4,857.0,Polycrystalline,Elastic Properties of Polycrystalline Thulium ...,"(L0+, O)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,L2O3,166.6,898.0,Polycrystalline,Elastic Properties of Polycrystalline Thulium ...,"(L0+, O)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
dat_t_sgd= dat_t_sgd.dropna()

In [28]:
# exclude some columns
y= dat_t_sgd['Elastic Modulus (GPa)']
excluded= ['Formula','Elastic Modulus (GPa)','Crystallinity','composition','Reference']
X= dat_t_sgd.drop(excluded,axis=1)
print('Number of descriptors: %i'%len(X.columns))
print('Number of data points: %i'%X.shape[0])

Number of descriptors: 138
Number of data points: 875


In [29]:
# split data set
X_learn= X[:475]
y_learn= y[:475]

X_test= X[475:]
y_test= y[475:]

## SGD Regressor

In [4]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error

sgd= SGDRegressor()

In [30]:
# Try fitting
sgd.fit(X_learn,y_learn)

/Users/Prabudhya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [32]:
# get fit statistics
print('training R2 = ' + str(round(sgd.score(X, y), 3)))
print('training RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y_test, y_pred=sgd.predict(X_test))))

training R2 = -7.927382926201711e+26
training RMSE = 3542895998469434.500


lol ... will come back and work on this

## Random Forest

In [33]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=50, random_state=1)

rf.fit(X_learn, y_learn)
print('training R2 = ' + str(round(rf.score(X, y), 3)))
print('training RMSE = %.3f' % np.sqrt(mean_squared_error(y_true=y_test, y_pred=rf.predict(X_test))))

training R2 = 0.928
training RMSE = 57.428


ok this is better! I suppose the predictions are off by 57.428 GPa